## doubledot.SFMD
> class to add fields to SF objects

Are we attempting test first coding? yes?

## Get field description for SF object

necessary for progressive testing   

[Salesforce curl example](https://developer.salesforce.com/docs/atlas.en-us.api_rest.meta/api_rest/dome_sobject_describe.htm)

```curl https://MyDomainName.my.salesforce.com/services/data/v57.0/sobjects/Account/describe/ -H "Authorization: Bearer token"```

In [6]:
from doubledot import *

In [16]:
# can I append a method to a class from a different module?

import io
from nbdev.showdoc import *
import requests
import json
import jmespath as jp
import re
from time import sleep
from fastcore.basics import patch
from fastcore.test import *
import fileinput
import pandas as pd
import os
from doubledot import ATMS_api
import time
import operator

In [35]:
sf = Salesforce()


Directory 'atms_download' already exists.


In [ ]:
atms = ATMS_api()
sf.atms = atms

### first hack

In [9]:
obj = "Contact"
sf_headers = { 'Authorization': f"Bearer {sf._sf_access_token}", 'Content-Type': 'application/json' }
url = f"https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/sobjects/{obj}/describe"
print(url)
response = requests.request("GET", url, headers=sf_headers)
print(response)
r = response.json()
r


https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/sobjects/Contact/describe
<Response [200]>


{'actionOverrides': [],
 'activateable': False,
 'associateEntityType': None,
 'associateParentEntity': None,
 'childRelationships': [{'cascadeDelete': True,
   'childSObject': 'AIInsightValue',
   'deprecatedAndHidden': False,
   'field': 'SobjectLookupValueId',
   'junctionIdListNames': [],
   'junctionReferenceTo': [],
   'relationshipName': None,
   'restrictedDelete': False},
  {'cascadeDelete': True,
   'childSObject': 'AIRecordInsight',
   'deprecatedAndHidden': False,
   'field': 'TargetId',
   'junctionIdListNames': [],
   'junctionReferenceTo': [],
   'relationshipName': None,
   'restrictedDelete': False},
  {'cascadeDelete': False,
   'childSObject': 'AcceptedEventRelation',
   'deprecatedAndHidden': False,
   'field': 'RelationId',
   'junctionIdListNames': [],
   'junctionReferenceTo': [],
   'relationshipName': 'AcceptedEventRelations',
   'restrictedDelete': False},
  {'cascadeDelete': True,
   'childSObject': 'AccountContactRole',
   'deprecatedAndHidden': False,
   'f

In [10]:
jp.search("fields[?type=='reference'].name",r)

['MasterRecordId',
 'AccountId',
 'ReportsToId',
 'OwnerId',
 'CreatedById',
 'LastModifiedById',
 'IndividualId']

In [11]:
jp.search("fields[].name",r)

['Id',
 'IsDeleted',
 'MasterRecordId',
 'AccountId',
 'LastName',
 'FirstName',
 'Salutation',
 'Name',
 'OtherStreet',
 'OtherCity',
 'OtherState',
 'OtherPostalCode',
 'OtherCountry',
 'OtherLatitude',
 'OtherLongitude',
 'OtherGeocodeAccuracy',
 'OtherAddress',
 'MailingStreet',
 'MailingCity',
 'MailingState',
 'MailingPostalCode',
 'MailingCountry',
 'MailingLatitude',
 'MailingLongitude',
 'MailingGeocodeAccuracy',
 'MailingAddress',
 'Phone',
 'Fax',
 'MobilePhone',
 'HomePhone',
 'OtherPhone',
 'AssistantPhone',
 'ReportsToId',
 'Email',
 'Title',
 'Department',
 'AssistantName',
 'LeadSource',
 'Birthdate',
 'Description',
 'OwnerId',
 'CreatedDate',
 'CreatedById',
 'LastModifiedDate',
 'LastModifiedById',
 'SystemModstamp',
 'LastActivityDate',
 'LastCURequestDate',
 'LastCUUpdateDate',
 'LastViewedDate',
 'LastReferencedDate',
 'EmailBouncedReason',
 'EmailBouncedDate',
 'IsEmailBounced',
 'PhotoUrl',
 'Jigsaw',
 'JigsawContactId',
 'CleanStatus',
 'IndividualId',
 'Level_

In [36]:
@patch
def get_fields(self:Salesforce, 
               obj:str # the name of the Salesforce object
               ) -> requests.Response:
    """Get the fields for a given Salesforce object"""
    sf_headers = { 'Authorization': f"Bearer {sf._sf_access_token}", 'Content-Type': 'application/json' }
    url = f"https://cremaconsulting-dev-ed.develop.my.salesforce.com/services/data/v57.0/sobjects/{obj}/describe"
    # print(url)
    response = requests.request("GET", url, headers=sf_headers)
    print(response)
    r = response.json()
    if response.status_code == 200:
        r = response.json()
        names = jp.search("fields[].name",r)
        return names
    else:
        raise Exception(f"Error: {response.status_code} {response.reason}")

In [37]:
# operator.contains(a,b) == b in a  
test(  sf.get_fields("Contact"),"External_Id__c", operator.contains )

<Response [200]>


In [44]:
# operator.contains(a,b) == b in a  
[f for f in sf.get_fields("MembershipTerm__c") if re.match("contact", f, re.IGNORECASE)]

<Response [200]>


[]

In [47]:
atms.retrieve_and_clean('sales', since_date='2021-01-01', max_rows=100)

download dir is:  /Users/josephmann/Documents/Github/doubledot/atms_download/lwtyysdr
resp_d = self.get_telus_data(sales,offset=0, count= 100, since_date=2021-01-01)
ATMS_api.get_telus_data: since_date is 2021-01-01
http://crm-api-telus.atmsplus.com/api/sales/lastupdate?count=100&offset=0&updateDate=2021-01-01
resp_d.keys():  dict_keys(['response', 'done'])
done: False, resp_d.json() : <Response [200]>
cleaning_data_file - download dir is:  /Users/josephmann/Documents/Github/doubledot/atms_download/lwtyysdr
creating file:  /Users/josephmann/Documents/Github/doubledot/atms_download/lwtyysdr/atms_transformed_sales.json
Finished cleaning atms_sales.json -> /Users/josephmann/Documents/Github/doubledot/atms_download/lwtyysdr/atms_transformed_sales.json
ATMS_api - Attempting to load:  /Users/josephmann/Documents/Github/doubledot/atms_download/lwtyysdr/atms_transformed_sales.json  into dict
ATMS_api: loaded 100 sales into dict


In [77]:
# sales fields 
jp.search("[0] | [keys(@), length(@)]", _d)

[['saleKey',
  'saleAmount',
  'paymentAmount',
  'saleDate',
  'active',
  'terminalKey',
  'ticketCount',
  'eventDate',
  'booking',
  'saleComment',
  'saleDetails',
  'tickets'],
 12]

In [78]:
# sales_booking
_d = json.load(open("atms_download/lwtyysdr/atms_sales.json"))
jp.search("[0].booking", _d)

{'bookingId': 609138,
 'bookingContactKey': 1156341,
 'bookingContactType': 'Primary                                 ',
 'contactKey': 469651,
 'contactIndividualKey': 459024,
 'contactOrganizationKey': 0,
 'displayName': 'Visscher, Whitney',
 'firstName': 'Whitney',
 'lastName': 'Visscher',
 'email': 'whitneyvisscher@gmail.com',
 'phone': '7806609317'}

In [116]:
# sales_booking

## from 10th list item extract "tickets" list and then the first item in that list
key_l = jp.search("keys([10].tickets[0])", _d)
search_s = ', '.join(f"{k}: [type({k}), {k}] " for k in key_l)
search_s = "[20:30].tickets[].{" + search_s + " }"
print(search_s)
jp.search(search_s, _d)

[20:30].tickets[].{ticketKey: [type(ticketKey), ticketKey] , saleKey: [type(saleKey), saleKey] , saleDetailKey: [type(saleDetailKey), saleDetailKey] , itemKey: [type(itemKey), itemKey] , itemDescription: [type(itemDescription), itemDescription] , itemShort: [type(itemShort), itemShort] , itemType: [type(itemType), itemType] , scheduleKey: [type(scheduleKey), scheduleKey] , scheduleDate: [type(scheduleDate), scheduleDate] , facilityKey: [type(facilityKey), facilityKey] , facilityDescription: [type(facilityDescription), facilityDescription] , facilityShort: [type(facilityShort), facilityShort] , ticketText: [type(ticketText), ticketText] , rate: [type(rate), rate] , category: [type(category), category] , cancelled: [type(cancelled), cancelled] , redeemed: [type(redeemed), redeemed] , refunded: [type(refunded), refunded] , isValid: [type(isValid), isValid] , validationText: [type(validationText), validationText] , redeemedCount: [type(redeemedCount), redeemedCount] , ticketDetailKey: [typ

[{'ticketKey': ['number', 4213604],
  'saleKey': ['number', 1943407],
  'saleDetailKey': ['number', 6512140],
  'itemKey': ['number', 750],
  'itemDescription': ['string', '100 Gift Card'],
  'itemShort': ['null', None],
  'itemType': ['null', None],
  'scheduleKey': ['number', 0],
  'scheduleDate': ['string', '0001-01-01T00:00:00'],
  'facilityKey': ['number', 0],
  'facilityDescription': ['null', None],
  'facilityShort': ['null', None],
  'ticketText': ['null', None],
  'rate': ['string', 'Item'],
  'category': ['string', 'Item'],
  'cancelled': ['null', None],
  'redeemed': ['null', None],
  'refunded': ['null', None],
  'isValid': ['boolean', False],
  'validationText': ['null', None],
  'redeemedCount': ['number', 0],
  'ticketDetailKey': ['number', 4213711],
  'scheduleDetailKey': ['number', 0],
  'seats': ['number', 0],
  'seatsSold': ['number', 0],
  'totalTicketsScanned': ['number', 0],
  'scheduleEndDate': ['string', '0001-01-01T00:00:00'],
  'ticketStatus': ['string', ''],


In [110]:
jp.search("[20:30].tickets[*].{ticketKey: type(ticketKey)}", _d)

[[], [], [], [], [{'ticketKey': 'number'}], [], [], [], [], []]

In [130]:
jp.search("[20:30].saleDetails[].saleDetailId", _d)

[6512134, 6512135, 6512136, 6512137, 6512138, 6512140, 6512141, 6512142]

In [136]:
jp.search("[20:40].tickets[].itemDescription", _d)

['100 Gift Card']

In [91]:
sales_keys_l = jp.search("keys([0])",_d)
search_s = ', '.join(f"{k}: type({k}) " for k in sales_keys_l)
search_s = "[40:48].{" + search_s + " }"
print(search_s)
jp.search(search_s, _d)

[40:48].{saleKey: type(saleKey) , saleAmount: type(saleAmount) , paymentAmount: type(paymentAmount) , saleDate: type(saleDate) , active: type(active) , terminalKey: type(terminalKey) , ticketCount: type(ticketCount) , eventDate: type(eventDate) , booking: type(booking) , saleComment: type(saleComment) , saleDetails: type(saleDetails) , tickets: type(tickets)  }


[{'saleKey': 'string',
  'saleAmount': 'string',
  'paymentAmount': 'string',
  'saleDate': 'string',
  'active': 'boolean',
  'terminalKey': 'number',
  'ticketCount': 'number',
  'eventDate': 'null',
  'booking': 'object',
  'saleComment': 'null',
  'saleDetails': 'array',
  'tickets': 'array'},
 {'saleKey': 'string',
  'saleAmount': 'string',
  'paymentAmount': 'string',
  'saleDate': 'string',
  'active': 'boolean',
  'terminalKey': 'number',
  'ticketCount': 'number',
  'eventDate': 'null',
  'booking': 'object',
  'saleComment': 'null',
  'saleDetails': 'array',
  'tickets': 'array'},
 {'saleKey': 'string',
  'saleAmount': 'string',
  'paymentAmount': 'string',
  'saleDate': 'string',
  'active': 'boolean',
  'terminalKey': 'number',
  'ticketCount': 'number',
  'eventDate': 'null',
  'booking': 'object',
  'saleComment': 'null',
  'saleDetails': 'array',
  'tickets': 'array'},
 {'saleKey': 'string',
  'saleAmount': 'string',
  'paymentAmount': 'string',
  'saleDate': 'string',
 